# Note
* 直接存
    * property_items = ['securities_property', 'index_property']
    * others = ['futures', 'index_components', 'index']

- daily -> days=15
    - daily_items = ['securities_trading_data', 'securities_returns', 'margin_trading', 'institutional_investors']

- monthly -> months=2
    - monthly_items = ['monthly_revenue']

- quarterly -> months=6
    - quarterly_items = ['financial_report']

In [1]:
import pandas as pd
from datetime import datetime
import json

import os, sys
sys.path.extend(['../', '../../'])
from Data_Loader.api_connecter.tej_handler import TEJHandler

In [1]:

# 換地點要改
CONFIG_ROOT = os.path.join(os.path.dirname(os.getcwd()), 'config')

# read data
import configparser
config = configparser.ConfigParser()
for file_name in os.listdir(CONFIG_ROOT):
    file_path = os.path.join(CONFIG_ROOT, file_name)
    if os.path.isfile(file_path):
        if file_name.endswith('.ini'):
            config.read(file_path)
SAVE_ROOT = config.get('path', 'alpha_data_path')


# Update

In [3]:
# (v)property
sort_columns = ['目前狀態', '代碼']
datetime_column = '目前狀態'
start_date = datetime.today()-pd.DateOffset(years=1)

## get property from different tej database
new_data_index_property = (
    TEJHandler(exchange='tej', symbol_type='index_property')
    .fetch_data(start_date = start_date)
)
new_data_securities_property = (
    TEJHandler(exchange='tej', symbol_type='securities_property')
    .fetch_data(start_date = start_date)
)

## concat
new_data = (
    pd.concat([new_data_index_property, new_data_securities_property], axis=0)
    # 證券、指數代碼不同 -> 可以不設 index 並 axis=0 來 concat
    .drop_duplicates(subset=sort_columns, keep='last')
    .sort_values(by=sort_columns)
)

## update
update_parquet_data(
    data_path = os.path.join(SAVE_ROOT, f"{'property'}.parquet"),
    new_data = new_data,
    datetime_column = datetime_column
)

NameError: name 'update_parquet_data' is not defined

In [40]:
# (v)index
symbol_name_dict = (
    pd.read_parquet(os.path.join(SAVE_ROOT, f"{'property'}.parquet"))
    .set_index('代碼')['名稱(中文)']
    .to_dict()
)
handler = TEJHandler(exchange='tej', symbol_type='index')
new_data = (handler.fetch_data(start_date = datetime.today()-pd.DateOffset(days=15)))
new_data['名稱(中文)'] = new_data['symbol'].map(symbol_name_dict)

update_parquet_data(
    data_path = os.path.join(SAVE_ROOT, f"{'index'}.parquet"),
    new_data = new_data,
    datetime_column = handler.columns_rename_dict[handler.params_name['datetime']]
)

In [41]:
# (v)futures
handler = TEJHandler(exchange='tej', symbol_type='futures')
new_data = (handler.fetch_data(start_date = datetime.today()-pd.DateOffset(days=15)))

for col in ['報酬率', '標的現貨報酬率']:
    new_data[col] = new_data[col] / 100

update_parquet_data(
    data_path = os.path.join(SAVE_ROOT, f"{'futures'}.parquet"),
    new_data = new_data,
    datetime_column = handler.columns_rename_dict[handler.params_name['datetime']]
)

In [31]:
# df = pd.read_table(os.path.join(SAVE_ROOT, f"{'futures'}.txt"), encoding='cp950')
# del df['簡稱']

# df = handler.clean_data(
#     data = df,
#     datetime_cols = ['日期', '到期月'],
#     str_cols = ['期貨名稱', '標的證券'],
# )

# df['標的現貨代碼'] = df['標的證券'].str.split('  ').str[0]
# df['標的現貨名稱'] = df['標的證券'].str.split('  ').str[1]

# df = df.rename(columns={
#     '期貨名稱': 'symbol',
#     '日期': 'datetime',
#     '成交張數(量)': '成交量',
#     '標的證券ROI%': '標的現貨報酬率',
# })

# for col in ['報酬率', '標的現貨報酬率']:
#     df[col] = df[col] / 100

# # df[new_data.columns].sort_values(by=['datetime', 'symbol', '到期月']).to_parquet(os.path.join(SAVE_ROOT, f"{'futures'}.parquet"))

c:\Users\hottari\Collections\Trading_System\Data_Loader\api_connecter\../..\prepare_data.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data[col] = pd.to_datetime(data[col]).dt.tz_localize(None)


In [42]:
# (v)index_components
handler = TEJHandler(exchange='tej', symbol_type='index_components')
new_data = (handler.fetch_data(start_date = datetime.today()-pd.DateOffset(days=15)))

for col in ['前日市值比重']:
    new_data[col] = new_data[col] / 100

index_name_dict = {
    'Y9999': '加權指數', 
    'TWN50': '台灣50指數', 
    'ZGT50': '富櫃50指數',
}
new_data['指數名稱'] = new_data['index_symbol'].map(index_name_dict)
new_data['symbol'] = new_data['成份股'].str.split(' ').str[0]


update_parquet_data(
    data_path = os.path.join(SAVE_ROOT, f"{'index_components'}.parquet"),
    new_data = new_data,
    datetime_column = handler.columns_rename_dict[handler.params_name['datetime']]
)

## pdata

In [119]:
# (
#     pd.read_parquet(os.path.join(SAVE_ROOT, f"{'pdata_daily'}.parquet"))
#     .rename(columns={'日期': 'datetime', '證券碼': 'symbol'})
#     .to_parquet(os.path.join(SAVE_ROOT, f"{'pdata_daily'}.parquet"))
# )

In [43]:
# (v)daily
sort_columns = ['datetime', 'symbol']
datetime_column = 'datetime'
start_date = datetime.today()-pd.DateOffset(days=15)

## get property from different tej database
new_data_securities_trading_data = (
    TEJHandler(exchange='tej', symbol_type='securities_trading_data')
    .fetch_data(start_date = start_date)
    .sort_values(by=sort_columns)
    .drop_duplicates(subset=sort_columns, keep='last')
).set_index(sort_columns)

new_data_securities_returns = (
    TEJHandler(exchange='tej', symbol_type='securities_returns')
    .fetch_data(start_date = start_date)
    .sort_values(by=sort_columns)
    .drop_duplicates(subset=sort_columns, keep='last')
).set_index(sort_columns)

new_data_margin_trading = (
    TEJHandler(exchange='tej', symbol_type='margin_trading')
    .fetch_data(start_date = start_date)
    .sort_values(by=sort_columns)
    .drop_duplicates(subset=sort_columns, keep='last')
).set_index(sort_columns)

new_data_institutional_investors = (
    TEJHandler(exchange='tej', symbol_type='institutional_investors')
    .fetch_data(start_date = start_date)
    .sort_values(by=sort_columns)
    .drop_duplicates(subset=sort_columns, keep='last')
).set_index(sort_columns)


## concat
new_data = (
    pd.concat([
        new_data_securities_trading_data, 
        new_data_securities_returns,
        new_data_margin_trading,
        new_data_institutional_investors,
    ], axis=1)
    .reset_index()
    .sort_values(by=sort_columns)
)


## update
update_parquet_data(
    data_path = os.path.join(SAVE_ROOT, f"{'pdata_daily'}.parquet"),
    new_data = new_data,
    datetime_column = datetime_column
)


In [44]:
# MIND: 公司依據 資料日期 更新

# (v)monthly
sort_columns = ['資料日期', '證券碼', '營收發布日']
duplicates_columns = ['資料日期', '證券碼']
handler = TEJHandler(exchange='tej', symbol_type='monthly_revenue')
new_data = (
    handler.fetch_data(start_date = datetime.today()-pd.DateOffset(months=2))
    .sort_values(by=sort_columns)
    .drop_duplicates(subset=duplicates_columns, keep='last')
)

update_parquet_data(
    data_path = os.path.join(SAVE_ROOT, f"{'pdata_monthly'}.parquet"),
    new_data = new_data,
    datetime_column = handler.columns_rename_dict[handler.params_name['datetime']]
)

In [45]:
# MIND: 公司依據 財務資料日 更新

# (v)quarterly
sort_columns = ['財務資料日', '證券碼', '財報發布日']
duplicates_columns = ['財務資料日', '證券碼']
handler = TEJHandler(exchange='tej', symbol_type='financial_report')
new_data = (
    handler.fetch_data(start_date = datetime.today()-pd.DateOffset(months=6))
    .sort_values(by=sort_columns)
    .drop_duplicates(subset=duplicates_columns, keep='last')
)

update_parquet_data(
    data_path = os.path.join(SAVE_ROOT, f"{'pdata_quarterly'}.parquet"),
    new_data = new_data,
    datetime_column = handler.columns_rename_dict[handler.params_name['datetime']]
)

## pmart

In [46]:
def conduct_release_filter(number_of_listing_and_otc, data_unstack, release_unstack, signal_delay=0.9):
    # 只保留上市櫃的公佈日當基數來決定公佈比率
    data_unstack = data_unstack.loc[release_unstack.index].copy()
    # release filter
    release_rank_pct = release_unstack.rank(axis='columns', pct=True)
    release_filter = (release_rank_pct <= signal_delay)
    release_filter_index = release_unstack[release_filter].max(axis='columns').values
    # min release number checking 
    min_release_number = int(number_of_listing_and_otc * signal_delay)
    release_number = release_unstack.iloc[-1].dropna().shape[0]
    # conduct filter
    # 用 經過上市櫃篩選後之公佈日篩選器 (filter_condition) 進行兩種篩選
    filter_condition = release_filter
    data = (
        data_unstack[filter_condition]
        .set_index(release_filter_index).rename_axis('datetime')
        .dropna(axis=0, how='all')
        .dropna(axis=1, how='all')
        .sort_index()
    )
    # 公佈家數 < signal_delay 成數時，用前一期資料當最近一期資料
    if release_number < min_release_number:
        data.iloc[-1] = data.iloc[-2]
    return data


def conduct_listing_and_otc_filter(data_unstack, filter_condition):
    data = (
        data_unstack[filter_condition]
        .dropna(axis=0, how='all').dropna(axis=1, how='all')
        .copy()
    )
    return data


def align_with_trading_days(data, daily_index):
    """
    用 TAIEX datetime 當交易日 datetime
    """
    result = data.reindex(daily_index, method='ffill')
    return result

In [47]:
def get_daily_trading_datetime_index(start_date=None):
    df = get_parquet_data(
        data_path = os.path.join(SAVE_ROOT, f"{'index'}.parquet"),
        datetime_col = 'datetime',
        need_cols = ['datetime', 'symbol'],
        start_date = start_date,
    )
    daily_trading_datetime_index = df[df['symbol']=='IX0001'].sort_values(by='datetime')['datetime']
    return daily_trading_datetime_index



def get_condition_listing_and_otc(daily_trading_datetime_index, start_date=None):
    security_properties = get_parquet_data(
        data_path = os.path.join(SAVE_ROOT, f"{'property'}.parquet"),
        datetime_col = '目前狀態',
        start_date = start_date,
        need_cols = None,
    )
    # 篩選曾上市 ( 這邊上市日指上市或上櫃 )
    df_listing_and_otc = security_properties[security_properties['上市日'].isna()==False]
    df_listing_and_otc = df_listing_and_otc.set_index(['上市日', '代碼']).sort_index()
    df_listing_and_otc.index.names = ['datetime', 'symbol']
    # KY is not filtered, but a field is retained for identification
    df_listing_and_otc['is_KY'] = df_listing_and_otc['名稱(中文)'].str.contains('KY')

    # 篩選曾上市
    condition_ever_listing_and_otc = (
        (df_listing_and_otc['證券種類代碼']=='STOCK')
        & ((df_listing_and_otc['市場別']=='TSE') | (df_listing_and_otc['市場別']=='OTC') | (df_listing_and_otc['市場別']=='DIST')) 
        & (df_listing_and_otc['證券種類名稱']=='普通股')
    ).unstack().ffill()

    # 找出已下市
    delistings = df_listing_and_otc['下市日'].unstack().ffill()
    condition_delisting = delistings.le(delistings.index, axis=0)

    # 曾上市 且 未下市 = 上下市(櫃)
    condition_listing_and_otc = condition_ever_listing_and_otc & (condition_delisting==False)
    condition_listing_and_otc_daily = condition_listing_and_otc.reindex(daily_trading_datetime_index, method='ffill')

    # True = 1, False = 0, 相加 = True 的數量
    number_of_listing_and_otc = condition_listing_and_otc_daily.iloc[-1].dropna().sum()


    # 本月初的財務資料 datetime index 會在月初 -> filter 要拿上個月底(resample之後)的資料 shift(1) 變成本月初
    condition_listing_and_otc_for_monthly_quarterly = condition_listing_and_otc_daily.resample('ME').last().shift(1)
    # 更改為月初
    condition_listing_and_otc_for_monthly_quarterly.index = condition_listing_and_otc_for_monthly_quarterly.index.to_period('M').to_timestamp()
    # fillna(False) 強制 False 避免 Nan 值導致篩選錯誤 -> 須留意此時第一筆資料全部都變成 False (因為 shift(1))
    condition_listing_and_otc_for_monthly_quarterly = condition_listing_and_otc_for_monthly_quarterly.fillna(False)

    results = {
        'number_of_listing_and_otc': number_of_listing_and_otc,
        'condition_listing_and_otc': condition_listing_and_otc,
        'condition_listing_and_otc_daily': condition_listing_and_otc_daily,
        'condition_listing_and_otc_for_monthly_quarterly': condition_listing_and_otc_for_monthly_quarterly,
    }

    return results
    

In [48]:
start_date = datetime.today()-pd.DateOffset(months=9)
daily_trading_datetime_index = get_daily_trading_datetime_index(start_date)

# 上市櫃要全拿, 避免沒抓到過去的上市櫃資訊
listing_and_otc_results = get_condition_listing_and_otc(daily_trading_datetime_index)

C:\Users\medina\AppData\Local\Temp\ipykernel_6608\3022544382.py:32: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ).unstack().ffill()
C:\Users\medina\AppData\Local\Temp\ipykernel_6608\3022544382.py:51: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  condition_listing_and_otc_for_monthly_quarterly = condition_listing_and_otc_for_monthly_quarterly.fillna(False)


In [112]:
# (
#     pd.read_parquet(os.path.join(SAVE_ROOT, f"{'pdata_daily'}.parquet"))
#     .rename(columns={'日期': 'datetime', '證券碼': 'symbol'})
#     .to_parquet(os.path.join(SAVE_ROOT, f"{'pdata_daily'}.parquet"))
# )

In [49]:
# step 1: fetch pdata
pdata_daily = get_parquet_data(
    data_path = os.path.join(SAVE_ROOT, f"{'pdata_daily'}.parquet"),
    datetime_col = 'datetime',
    start_date = start_date,
    need_cols = None,
)

## 這邊用所屬年月撈比較穩 (公布日當日期篩選條件可能會缺所屬年月)
pdata_monthly = (
    get_parquet_data(
        data_path = os.path.join(SAVE_ROOT, f"{'pdata_monthly'}.parquet"),
        datetime_col = '資料日期',
        start_date = start_date,
        need_cols = None,   
    )
    #.sort_values(by=['資料日期', '證券碼', '營收發布日']).drop_duplicates(subset=['資料日期', '證券碼'], keep='last')
)
pdata_quarterly = (
    get_parquet_data(
        data_path = os.path.join(SAVE_ROOT, f"{'pdata_quarterly'}.parquet"),
        datetime_col = '財務資料日',
        start_date = start_date,
        need_cols = None,
    )
    #.sort_values(by=['財務資料日', '證券碼', '財報發布日']).drop_duplicates(subset=['財務資料日', '證券碼'], keep='last')
)



In [50]:
# step 2: form pmart

daily_mart = conduct_listing_and_otc_filter(
        data_unstack = pdata_daily.set_index(['datetime', 'symbol']).unstack(),
        filter_condition = listing_and_otc_results['condition_listing_and_otc_daily']
    ).stack(future_stack=True)

In [51]:
# Monthly and Quarterly
relaese_name_monthkly = '營收發布日'
relaese_name_quarterly = '財報發布日'
pdata_monthly = pdata_monthly.set_index(['資料日期', '證券碼']).rename_axis(['datetime', 'symbol'])
pdata_quarterly = pdata_quarterly.set_index(['財務資料日', '證券碼']).rename_axis(['datetime', 'symbol'])

## 利用上市櫃篩選後的公告日當篩選器
release_unstack_listing_and_otc_monthly = conduct_listing_and_otc_filter(
    data_unstack = pdata_monthly[relaese_name_monthkly].unstack(), 
    filter_condition = listing_and_otc_results['condition_listing_and_otc_for_monthly_quarterly']
)
release_unstack_listing_and_otc_quarterly = conduct_listing_and_otc_filter(
    data_unstack = pdata_quarterly[relaese_name_quarterly].unstack(), 
    filter_condition = listing_and_otc_results['condition_listing_and_otc_for_monthly_quarterly']
)

## monthly
monthly_mart = align_with_trading_days(
    data = conduct_release_filter(
        number_of_listing_and_otc = listing_and_otc_results['number_of_listing_and_otc'],
        data_unstack = pdata_monthly[[col for col in pdata_monthly.columns if col != relaese_name_monthkly]].unstack(),
        release_unstack = release_unstack_listing_and_otc_monthly,
        signal_delay = 0.9,
    ),
    daily_index = daily_trading_datetime_index,
).stack(future_stack=True)

## quarterly
quarterly_mart = align_with_trading_days(
    data = conduct_release_filter(
        number_of_listing_and_otc = listing_and_otc_results['number_of_listing_and_otc'],
        data_unstack = pdata_quarterly[[col for col in pdata_quarterly.columns if col != relaese_name_quarterly]].unstack(),
        release_unstack = release_unstack_listing_and_otc_quarterly,
        signal_delay = 0.9,
    ),
    daily_index = daily_trading_datetime_index,
).stack(future_stack=True)

In [52]:
updated_start = quarterly_mart.dropna().index.get_level_values('datetime')[0]
# 季資料的時間通常會是最晚開始的

sort_columns = ['datetime', 'symbol']
duplicates_columns = ['datetime', 'symbol']
new_data = (
    pd.concat([
        daily_mart[updated_start:], 
        monthly_mart[updated_start:], 
        quarterly_mart[updated_start:],
    ], axis='columns')
    .reset_index()
    .sort_values(by=sort_columns)
    .drop_duplicates(subset=duplicates_columns, keep='last')
)



In [53]:
update_parquet_data(
    data_path = os.path.join(SAVE_ROOT, f"{'pmart'}.parquet"),
    new_data = new_data,
    datetime_column = 'datetime'
)

### test start

In [56]:
new_data_co = pd.read_pickle(r'C:\Medina\alpha_portfolio_manager\labs\monitor\new_data_pmart_co.pkl')
new_data_test = new_data.set_index(['datetime', 'symbol']).sort_index()

In [58]:
index_diff = new_data_test.index.difference(new_data_co.index)

In [64]:
new_data_co.dropna(axis=0, how='all').dropna(axis=1, how='all')

融資買進(張)  融資賣出(張)  融券買入(張)  融券賣出(張)  融資餘額(張)  融券餘額(張)  \
datetime   symbol                                                         
2024-03-15 1101      223.0    160.0     18.0      0.0  17452.0      0.0   
           1102        3.0     38.0      0.0      0.0   1461.0      2.0   
           1103       93.0      1.0      0.0      0.0   2005.0      0.0   
           1104      165.0     30.0      0.0      0.0   1706.0      5.0   
           1108      465.0     25.0      0.0      0.0   2730.0      1.0   
...                    ...      ...      ...      ...      ...      ...   
2024-07-10 9951        0.0      0.0      0.0      0.0    441.0     15.0   
           9955       50.0     58.0      0.0      4.0   1821.0    385.0   
           9958      923.0    482.0     12.0     11.0  20789.0    628.0   
           9960        1.0      0.0      0.0      0.0     95.0      0.0   
           9962        3.0     11.0      0.0      0.0    819.0      0.0   

                     融資餘額(千元)   融券餘額(千元)  融資使用率  融券使用率  ...       營業利益  \
datetime   symbol                                       ...              
2024-03-15 1101     557970.11       0.00   0.92   0.00  ...  3433504.0   
           1102      60007.21      82.15   0.16   0.00  ...  1467155.0   
           1103      34574.62       0.00   1.01   0.00  ...   -20970.0   
           1104      53841.87     157.80   1.01   0.00  ...   309523.0   
           1108      44348.85      16.25   2.70   0.00  ...        NaN   
...                       ...        ...    ...    ...  ...        ...   
2024-07-10 9951      31523.21    1072.22   2.35   0.08  ...    63255.0   
           9955      63765.23   13481.39   7.06   1.49  ...   -33641.0   
           9958    5922665.52  178913.56  32.37   0.98  ...  1060381.0   
           9960       2612.78       0.00   1.13   0.00  ...    11872.0   
           9962      13359.77       0.00   3.63   0.00  ...     2778.0   

                   營業外收入及支出      所得稅費用      合併總損益  歸屬母公司淨利（損）     折舊－CFO  \
datetime   symbol                                                          
2024-03-15 1101    383504.0  1317094.0  2499914.0   1801908.0  2082429.0   
           1102    481242.0   667356.0  1281041.0   1325636.0  1124744.0   
           1103    -46157.0   -15624.0   -51503.0    -54360.0   110085.0   
           1104    228930.0    60119.0   478334.0    460561.0    46587.0   
           1108         NaN        NaN        NaN         NaN        NaN   
...                     ...        ...        ...         ...        ...   
2024-07-10 9951     50283.0    21400.0    92138.0     92138.0    30907.0   
           9955      2075.0        0.0   -31566.0    -31566.0    16630.0   
           9958     15803.0   215487.0   860697.0    650760.0   303801.0   
           9960     10323.0     4569.0    17626.0     17626.0     7647.0   
           9962      5508.0     1652.0     6634.0      6634.0     5554.0   

                     攤提－CFO   來自營運之現金流量   投資活動之現金流量  籌資活動之現金流量  
datetime   symbol                                               
2024-03-15 1101    282945.0  13796962.0 -11346930.0 -5820763.0  
           1102     80517.0   3335886.0  -7680917.0   693873.0  
           1103       671.0     84138.0   -214439.0    -2268.0  
           1104      1127.0    208391.0    -98470.0  -224833.0  
           1108         NaN         NaN         NaN        NaN  
...                     ...         ...         ...        ...  
2024-07-10 9951       581.0     21236.0   -205489.0   -81717.0  
           9955         0.0     19894.0    -54420.0    60503.0  
           9958       490.0    199138.0    322278.0  -632228.0  
           9960      2176.0     56976.0    -65795.0  -171890.0  
           9962       108.0    -10954.0    -45636.0    45397.0  

[145618 rows x 114 columns]

In [63]:
new_data_test.dropna(axis=0, how='all').dropna(axis=1, how='all')

融資買進(張)  融資賣出(張)  融券買入(張)  融券賣出(張)  融資餘額(張)  融券餘額(張)  \
datetime   symbol                                                         
2024-03-15 1101      223.0    160.0     18.0      0.0  17452.0      0.0   
           1102        3.0     38.0      0.0      0.0   1461.0      2.0   
           1103       93.0      1.0      0.0      0.0   2005.0      0.0   
           1104      165.0     30.0      0.0      0.0   1706.0      5.0   
           1108      465.0     25.0      0.0      0.0   2730.0      1.0   
...                    ...      ...      ...      ...      ...      ...   
2024-07-10 9951        0.0      0.0      0.0      0.0    441.0     15.0   
           9955       50.0     58.0      0.0      4.0   1821.0    385.0   
           9958      923.0    482.0     12.0     11.0  20789.0    628.0   
           9960        1.0      0.0      0.0      0.0     95.0      0.0   
           9962        3.0     11.0      0.0      0.0    819.0      0.0   

                     融資餘額(千元)   融券餘額(千元)  融資使用率  融券使用率  ...       營業利益  \
datetime   symbol                                       ...              
2024-03-15 1101     557970.11       0.00   0.92   0.00  ...  3433504.0   
           1102      60007.21      82.15   0.16   0.00  ...  1467155.0   
           1103      34574.62       0.00   1.01   0.00  ...   -20970.0   
           1104      53841.87     157.80   1.01   0.00  ...   309523.0   
           1108      44348.85      16.25   2.70   0.00  ...        NaN   
...                       ...        ...    ...    ...  ...        ...   
2024-07-10 9951      31523.21    1072.22   2.35   0.08  ...    63255.0   
           9955      63765.23   13481.39   7.06   1.49  ...   -33641.0   
           9958    5922665.52  178913.56  32.37   0.98  ...  1060381.0   
           9960       2612.78       0.00   1.13   0.00  ...    11872.0   
           9962      13359.77       0.00   3.63   0.00  ...     2778.0   

                   營業外收入及支出      所得稅費用      合併總損益  歸屬母公司淨利（損）     折舊－CFO  \
datetime   symbol                                                          
2024-03-15 1101    383504.0  1317094.0  2499914.0   1801908.0  2082429.0   
           1102    481242.0   667356.0  1281041.0   1325636.0  1124744.0   
           1103    -46157.0   -15624.0   -51503.0    -54360.0   110085.0   
           1104    228930.0    60119.0   478334.0    460561.0    46587.0   
           1108         NaN        NaN        NaN         NaN        NaN   
...                     ...        ...        ...         ...        ...   
2024-07-10 9951     50283.0    21400.0    92138.0     92138.0    30907.0   
           9955      2075.0        0.0   -31566.0    -31566.0    16630.0   
           9958     15803.0   215487.0   860697.0    650760.0   303801.0   
           9960     10323.0     4569.0    17626.0     17626.0     7647.0   
           9962      5508.0     1652.0     6634.0      6634.0     5554.0   

                     攤提－CFO   來自營運之現金流量   投資活動之現金流量  籌資活動之現金流量  
datetime   symbol                                               
2024-03-15 1101    282945.0  13796962.0 -11346930.0 -5820763.0  
           1102     80517.0   3335886.0  -7680917.0   693873.0  
           1103       671.0     84138.0   -214439.0    -2268.0  
           1104      1127.0    208391.0    -98470.0  -224833.0  
           1108         NaN         NaN         NaN        NaN  
...                     ...         ...         ...        ...  
2024-07-10 9951       581.0     21236.0   -205489.0   -81717.0  
           9955         0.0     19894.0    -54420.0    60503.0  
           9958       490.0    199138.0    322278.0  -632228.0  
           9960      2176.0     56976.0    -65795.0  -171890.0  
           9962       108.0    -10954.0    -45636.0    45397.0  

[145618 rows x 114 columns]

In [73]:
new_data_test.loc[
    new_data_test.dropna(axis=0, how='all').dropna(axis=1, how='all')[['流通股數(千股)']]
    .compare(new_data_co.dropna(axis=0, how='all').dropna(axis=1, how='all')[['流通股數(千股)']])
    .index
][['流通股數(千股)']]

流通股數(千股)
datetime   symbol           
2024-06-25 2427     196818.0
           2438      65196.0
           8040      96414.0
           9906      83971.0
2024-06-26 2427     196818.0
...                      ...
2024-07-05 6664      59311.0
           6933      41695.0
           8027     104073.0
           8069    1144891.0
           8092      56159.0

[214 rows x 1 columns]

In [76]:
diff_cols = new_data_test.dropna(axis=0, how='all').dropna(axis=1, how='all').compare(new_data_co.dropna(axis=0, how='all').dropna(axis=1, how='all')).columns.get_level_values(0).unique()

In [77]:
diff_cols

Index(['流通股數(千股)', '開盤價-除權息', '最高價-除權息', '最低價-除權息', '收盤價-除權息', '投信總持股率(%)',
       '自營總持股率(%)', '幣別'],
      dtype='object')

In [79]:
for col in diff_cols:
    print(col)
    display(
        new_data_test.dropna(axis=0, how='all').dropna(axis=1, how='all')[col]
        .compare(new_data_co.dropna(axis=0, how='all').dropna(axis=1, how='all')[col])
    )

流通股數(千股)


self      other
datetime   symbol                      
2024-06-25 2427     196818.0   196814.0
           2438      65196.0    60196.0
           8040      96414.0    68850.0
           9906      83971.0    83431.0
2024-06-26 2427     196818.0   196814.0
...                      ...        ...
2024-07-05 6664      59311.0    58047.0
           6933      41695.0    41666.0
           8027     104073.0   102736.0
           8069    1144891.0  1144432.0
           8092      56159.0    51821.0

[214 rows x 2 columns]

開盤價-除權息


self   other
datetime   symbol                  
2024-06-25 1219     18.0974   18.65
           1336     19.7519   20.15
           1410     37.5915   37.90
           1451     20.4737   21.00
           1452     16.6964   17.00
...                     ...     ...
2024-07-05 8183     41.8417   43.65
           8374    125.1741  125.50
           8905     30.1944   32.00
           8941     75.0026   77.00
           9905     23.5000   24.60

[1285 rows x 2 columns]

最高價-除權息


...

self  other
datetime   symbol              
2024-06-28 1582     1.58   1.59
           1815     0.90   0.91
           2439     3.12   3.29
           3010     1.97   1.99
           3042    10.63  10.66
           3491     2.25   2.26
           3533     7.05   7.08
           3661     5.65   5.69
           3665     8.02   8.47
           3708    10.24  10.62
           6187     7.73   7.75
           6664     0.31   0.32
           6670     2.22   2.24
           8112    19.42  19.50
           8996     6.49   6.55
2024-07-01 1815     0.90   0.91
           2439     3.15   3.32
           3010     2.69   2.71
           3661     5.65   5.69
           3665     8.02   8.47
           3708     9.82  10.19
           6187     7.73   7.75
           6415     2.55   2.56
           6664     0.31   0.32
2024-07-02 1815     0.90   0.91
           2439     3.15   3.32
           3010     4.27   4.31
           3665     8.16   8.61
           6664     0.31   0.32
2024-07-03 1436     0.37   0.38
           1815     0.90   0.91
           2439     3.63   3.82
           3010     5.08   5.12
           3665     9.10   9.60
           6664     0.31   0.32
2024-07-04 1436     0.40   0.41
           1815     0.90   0.91
           2439     3.52   3.71
           3010     5.70   5.75
           3665     9.30   9.81
           6664     0.31   0.32
2024-07-05 6664     0.31   0.32

自營總持股率(%)


self  other
datetime   symbol             
2024-06-25 8040    1.33   1.86
2024-06-26 8040    1.33   1.86
2024-06-27 8040    1.33   1.86
2024-06-28 3663    0.71   0.72
           3665    0.42   0.44
           4566    0.51   0.57
           6175    0.11   0.12
           6190    0.71   0.72
           8027    0.69   0.70
           8040    1.33   1.86
2024-07-01 3010    0.27   0.28
           3663    0.63   0.64
           3665    0.38   0.41
           3708    0.38   0.39
           4121    2.22   2.23
           4566    1.82   2.04
           8027    0.78   0.79
2024-07-02 3665    0.41   0.44
           4566    1.80   2.02
           8027    0.80   0.81
2024-07-03 3665    0.61   0.65
           4566    1.84   2.06
           6190    0.69   0.70
           8027    1.11   1.13
2024-07-04 3010    0.33   0.34
           3665    0.57   0.61
           4566    1.79   2.01
           8027    1.00   1.01
2024-07-05 4566    1.60   1.79
           8027    0.90   0.91

幣別


self other
datetime   symbol            
2024-05-14 1101    None   NTD
           1102    None   NTD
           1103    None   NTD
           1104    None   NTD
           1108     NaN   NTD
...                 ...   ...
2024-07-10 9951    None   NTD
           9955    None   NTD
           9958    None   NTD
           9960    None   NTD
           9962    None   NTD

[65887 rows x 2 columns]

In [71]:
(
    new_data_test.dropna(axis=0, how='all').dropna(axis=1, how='all')[['流通股數(千股)']]
    .compare(new_data_co.dropna(axis=0, how='all').dropna(axis=1, how='all')[['流通股數(千股)']])#.dropna(axis=0, how='all').dropna(axis=1, how='all')
)

流通股數(千股)           
                        self      other
datetime   symbol                      
2024-06-25 2427     196818.0   196814.0
           2438      65196.0    60196.0
           8040      96414.0    68850.0
           9906      83971.0    83431.0
2024-06-26 2427     196818.0   196814.0
...                      ...        ...
2024-07-05 6664      59311.0    58047.0
           6933      41695.0    41666.0
           8027     104073.0   102736.0
           8069    1144891.0  1144432.0
           8092      56159.0    51821.0

[214 rows x 2 columns]

In [61]:
new_data_test.loc[index_diff].dropna(axis=0, how='all').dropna(axis=1, how='all')

融資買進(張)  融資賣出(張)  融券買入(張)  融券賣出(張)  融資餘額(張)  融券餘額(張)  \
datetime   symbol                                                         
2024-07-03 3089        NaN      NaN      NaN      NaN      NaN      NaN   
           3682        NaN      NaN      NaN      NaN      NaN      NaN   
           4944        NaN      NaN      NaN      NaN      NaN      NaN   
           5281        NaN      NaN      NaN      NaN      NaN      NaN   
           8418        NaN      NaN      NaN      NaN      NaN      NaN   
2024-07-04 3089        NaN      NaN      NaN      NaN      NaN      NaN   
           3682        NaN      NaN      NaN      NaN      NaN      NaN   
           4944        NaN      NaN      NaN      NaN      NaN      NaN   
           5281        NaN      NaN      NaN      NaN      NaN      NaN   
           8418        NaN      NaN      NaN      NaN      NaN      NaN   
2024-07-05 3089        NaN      NaN      NaN      NaN      NaN      NaN   
           3682        NaN      NaN      NaN      NaN      NaN      NaN   
           4944        NaN      NaN      NaN      NaN      NaN      NaN   
           5281        NaN      NaN      NaN      NaN      NaN      NaN   
           8418        NaN      NaN      NaN      NaN      NaN      NaN   
2024-07-08 3089        NaN      NaN      NaN      NaN      NaN      NaN   
           3682        NaN      NaN      NaN      NaN      NaN      NaN   
           4944        NaN      NaN      NaN      NaN      NaN      NaN   
           5281        NaN      NaN      NaN      NaN      NaN      NaN   
           8418        NaN      NaN      NaN      NaN      NaN      NaN   
2024-07-09 3089        NaN      NaN      NaN      NaN      NaN      NaN   
           3682        NaN      NaN      NaN      NaN      NaN      NaN   
           4944        NaN      NaN      NaN      NaN      NaN      NaN   
           5281        NaN      NaN      NaN      NaN      NaN      NaN   
           8418        NaN      NaN      NaN      NaN      NaN      NaN   
2024-07-10 3089        NaN      NaN      NaN      NaN      NaN      NaN   
           3682        NaN      NaN      NaN      NaN      NaN      NaN   
           4944        NaN      NaN      NaN      NaN      NaN      NaN   
           5281        NaN      NaN      NaN      NaN      NaN      NaN   
           8418        NaN      NaN      NaN      NaN      NaN      NaN   

                   融資餘額(千元)  融券餘額(千元)  融資使用率  融券使用率  ...  營業利益  營業外收入及支出  \
datetime   symbol                                    ...                   
2024-07-03 3089         NaN       NaN    NaN    NaN  ...   NaN       NaN   
           3682         NaN       NaN    NaN    NaN  ...   NaN       NaN   
           4944         NaN       NaN    NaN    NaN  ...   NaN       NaN   
           5281         NaN       NaN    NaN    NaN  ...   NaN       NaN   
           8418         NaN       NaN    NaN    NaN  ...   NaN       NaN   
2024-07-04 3089         NaN       NaN    NaN    NaN  ...   NaN       NaN   
           3682         NaN       NaN    NaN    NaN  ...   NaN       NaN   
           4944         NaN       NaN    NaN    NaN  ...   NaN       NaN   
           5281         NaN       NaN    NaN    NaN  ...   NaN       NaN   
           8418         NaN       NaN    NaN    NaN  ...   NaN       NaN   
2024-07-05 3089         NaN       NaN    NaN    NaN  ...   NaN       NaN   
           3682         NaN       NaN    NaN    NaN  ...   NaN       NaN   
           4944         NaN       NaN    NaN    NaN  ...   NaN       NaN   
           5281         NaN       NaN    NaN    NaN  ...   NaN       NaN   
           8418         NaN       NaN    NaN    NaN  ...   NaN       NaN   
2024-07-08 3089         NaN       NaN    NaN    NaN  ...   NaN       NaN   
           3682         NaN       NaN    NaN    NaN  ...   NaN       NaN   
           4944         NaN       NaN    NaN    NaN  ...   NaN       NaN   
           5281         NaN       NaN    NaN    NaN  ...   NaN       NaN   
           8418       

### test tail

# API info

In [104]:
handler = TEJHandler()

In [105]:
handler.get_api_using_info()

,value
remain_req_day,1815
remain_rows_day,28316634
remain_rows_month,9223372036850489479
reqDayLimit,2000
rowsDayLimit,30000000
rowsMonthLimit,9223372036854775807
todayReqCount,185
todayRows,1683366
monthRows,4286328


In [ ]:
# df = handler.fetch_data()

# Tail